change datetime format

format reference: 1. https://github.com/d3/d3-format/blob/master/README.md
            2. https://mkaz.tech/code/python-string-format-cookbook/

In [ ]:
def updateDT(formt):
    stocks['layout'].update({'xaxis' : {'tickformat' : formt}})
    pyo.iplot(stocks)

updateDT("%d/%m/%y")

Hover text info change

In [ ]:
def updateHoverInfo(info):
    trace.update({'hoverinfo' : info})
    pyo.iplot([trace])
    
'''updateHoverInfo('text+x+y')'''

Create Stack Area with hover text 3.8

In [ ]:
def createStackedQuantArea(df, time, cols, hover, title, yaxisTitle):
    """
    A function which manipulates the data into the correct format to produce a stacked quantity area plot with Plotly.
    
    Takes five arguments:
    
    df - a pandas DataFrame
    time - the time element of the data, must be a column in the DataFrame
    cols - the name of the columns in the DataFrame which you want to include in the area plot
    title - the title of the chart
    yaxisTitle - the yaxis title of the chart (the xaxis title comes from the time variable)
    """
    traces = []
    stackedAreaDF = df.loc[:, ([time] + cols)]
    stackedAreaDF.fillna(0, inplace=True)
    
    cumulative = stackedAreaDF[cols].cumsum(axis = 1)
    
    cumulAndOrig = cumulative.merge(stackedAreaDF, 
                                         left_index = True,
                                         right_index = True,
                                        suffixes = ('_c','_o'))
    
    for col in cols:
        cumulAndOrig[col + '_t'] = "<b>" + str(col)[:-6]  + "</b><br>" + str(hover) + cumulAndOrig[col + "_o"].apply(lambda x:
            "{:,}Kt".format(int(round(x, 0))))
        
        traces.append({'type' : 'scatter',
                      'x' : cumulAndOrig[time],
                      'y' : cumulAndOrig[col + "_c"],
                       'text' : cumulAndOrig[col + "_t"],
                       'hoverinfo' : 'text+x',
                      'name' : col[:-6],
                      'mode' : 'lines',
                      'fill' : 'tonexty'})
        
    data = Data(traces)
    layout = {'title' : title,
             'xaxis' : {'title' : time},
             'yaxis' : {'title' : yaxisTitle,
                       'ticksuffix' : ' Kt'},
             'hovermode' : 'closest'}
    fig = Figure(data = data, layout = layout)
    pyo.iplot(fig)
    return cumulAndOrig,fig
    
'''
C02Quant = createStackedQuantArea(emissions, 'Year', ['United Arab Emirates | ARE','United Kingdom | GBR', 
                   'United States | USA','China | CHN', 'India | IND'], 'Total C02 Emissions: ',
                            "Quantity of Co2 Emissions, 1960-2011", 'Quantity of Co2 Emissions')
'''

Create proportional Stack Area with hover text 3.9

In [ ]:
def createStackedPropArea(df, time, cols, hover, title, yaxisTitle): 
    """
    A function which manipulates the data into the correct format to produce a stacked proportional area plot with Plotly.
    
    Takes five arguments:
    
    df - a pandas DataFrame
    time - the time element of the data, must be a column in the DataFrame
    cols - the name of the columns in the DataFrame which you want to include in the area plot
    hover - the text that you want to include on the hovertext
    title - the title of the chart
    yaxisTitle - the yaxis title of the chart (the xaxis title comes from the time variable)
    """
    PCcols = []
    traces = []
    
    stackedAreaDF = df.loc[:, ([time] + cols)]
    stackedAreaDF.fillna(0, inplace=True)
        
    stackedAreaDF['Total'] = stackedAreaDF[cols].sum(axis =1)
    
    for col in cols:
        stackedAreaDF["pc_"+str(col)] = stackedAreaDF[col] / stackedAreaDF['Total']
        PCcols.append("pc_"+str(col))
        
    stackedPCAreaDF = stackedAreaDF[PCcols].cumsum(axis=1)
    stackedAreaData = stackedAreaDF[PCcols + [time]].merge(stackedPCAreaDF[PCcols], 
                                        left_index = True,
                                         right_index = True,
                                        suffixes = ('_o','_c'))

    for col in PCcols:       
        stackedAreaData[col + '_t'] = "<b>" + str(col)[3:-6]  + "</b><br>" + str(hover) + stackedAreaData[col + "_o"].apply(lambda x:
            "{:.0%}".format(x))
        
        
        traces.append({'type' : 'scatter',
                      'x' : stackedAreaData[time],
                      'y' : stackedAreaData[col + "_c"],
                       'text' : stackedAreaData[col + "_t"],
                       'hoverinfo' : 'text+x',
                      'name' : col[3:-6],
                      'mode' : 'lines',
                      'fill' : 'tonexty'})
    
    data = Data(traces)
    layout = {'title' : title,
             'xaxis' : {'title' : time},
             'yaxis' : {'title' : yaxisTitle,
                       'tickformat' : '%'},
              'hovermode' : 'closest'}
    fig = Figure(data = data, layout = layout)
    pyo.iplot(fig)

    return stackedAreaData,fig
    
    
C02Prop = createStackedPropArea(emissions, 'Year', ['United Arab Emirates | ARE','United Kingdom | GBR', 
                   'United States | USA','China | CHN', 'India | IND',], 'Proportion of total C02 Emissions: ',
                            "Proportion of Co2 Emissions, 1960-2011", 'Proprtion of Co2 Emissions')

scatterplot Matrix without category 4.14 or create new col with single value "nocat", and then use scatterplot Matrix with category 4.14

In [ ]:
from plotly import tools
def scatterplotMatrix(df, scatterColumns,color, title):
    """
    This function create a scatterplot matrix and expects the following inputs:
    - df - The DataFrame which contains the data
    - scatterColumns - a list of the columns in the DataFrame which we want to plot on a scatterplot matrix
    - title - the title of the chart
    
    This function does not create a scatterplot where the same variable intersects with itself.
    """
    
    fig = tools.make_subplots(rows = len(scatterColumns),
                             cols = len(scatterColumns),
                             print_grid = True,
                             shared_xaxes = True,
                             shared_yaxes = True)
    
    diff = max(df[scatterColumns].max()) - min(df[scatterColumns].min())
    
    minimum = min(df[scatterColumns].min()) - (diff * 0.1)
    maximum = max(df[scatterColumns].max()) + (diff * 0.1)
    
    for i, column in enumerate(scatterColumns):
        fig['layout']['xaxis{}'.format(i + 1)].update({'title' : column,
                                                      'range' : [minimum,maximum]})
        
        for j, row in enumerate(scatterColumns):
            fig['layout']['yaxis{}'.format(i + 1)].update({'title' : row,
                                                      'range' : [minimum,maximum]})
            
            if column != row:
                if i == 0 and j == 1:
                    show = True
                else:
                    show = False
                
                    fig.append_trace({'type' : 'scatter',
                                     'mode' : 'markers',
                                     'x' : df.loc[:, column],
                                     'y' : df.loc[:, row],
                                     'marker' : {'color' : color,
                                                'size' : 3},
                                     'showlegend' : False},
                                    col = i + 1,
                                    row = j + 1)
                    
    fig['layout'].update({'title' : title,
                         'height' : len(scatterColumns * 200),
                         'width' : len(scatterColumns * 200)})
    pyo.iplot(fig)
    return fig
'''
irisScatter = scatterplotMatrix(iris, 
                  ['Sepal length','Sepal width','Petal length','Petal width'],
                 'purple','Scatterplot matrix of Iris dataset')
'''

scatterplot Matrix with category 4.14

In [ ]:
from plotly import tools
def scatterplotMatrix(df, scatterColumns, categoricalColumn, colours, title):
    """
    This function create a scatterplot matrix and expects the following inputs:
    - df - The DataFrame which contains the data
    - scatterColumns - a list of the columns in the DataFrame which we want to plot on a scatterplot matrix
    - categoricalColumn - the column which contains the categories of data which should be plotted
    - colours - a list of colours equal in length to the number of categories in the categoricalColumn
    - title - the title of the chart
    
    This function does not create a scatterplot where the same variable intersects with itself.
    """
    
    categories = list(df[categoricalColumn].unique())
    colourLookup = dict(zip(categories, colours))
    
    fig = tools.make_subplots(rows = len(scatterColumns),
                             cols = len(scatterColumns),
                             print_grid = True,
                             shared_xaxes = True,
                             shared_yaxes = True)
    
    diff = max(df[scatterColumns].max()) - min(df[scatterColumns].min())
    
    minimum = min(df[scatterColumns].min()) - (diff * 0.1)
    maximum = max(df[scatterColumns].max()) + (diff * 0.1)
    
    for i, column in enumerate(scatterColumns):
        fig['layout']['xaxis{}'.format(i + 1)].update({'title' : column,
                                                      'range' : [minimum,maximum]})
        
        for j, row in enumerate(scatterColumns):
            fig['layout']['yaxis{}'.format(i + 1)].update({'title' : row,
                                                      'range' : [minimum,maximum]})
            
            if column != row:
                if i == 0 and j == 1:
                    show = True
                else:
                    show = False
                
                for category, colour in colourLookup.items():
                    fig.append_trace({'type' : 'scatter',
                                     'mode' : 'markers',
                                     'x' : df.loc[df[categoricalColumn] == category, column],
                                     'y' : df.loc[df[categoricalColumn] == category, row],
                                     'marker' : {'color' : colour,
                                                'size' : 3},
                                     'name' : category,
                                     'legendgroup' : category,
                                     'showlegend' : show},
                                    col = i + 1,
                                    row = j + 1)
                    
    fig['layout'].update({'title' : title,
                         'height' : len(scatterColumns * 200),
                         'width' : len(scatterColumns * 200)})
    pyo.iplot(fig)
    return fig
'''scatterplotMatrix(iris, 
                  ['Sepal length','Sepal width','Petal length','Petal width'], 
                  'Species', 
                  ['purple','orange','green'],
                 'Scatterplot matrix of Iris dataset')
                 '''
